In [ ]:
from pyvis.network import Network
import networkx as nx

G = nx.Graph()
G.add_edge("chat", "animal")
G.add_edge("chat", "mignon")

net = Network()
net.from_nx(G)
net.show("graph.html")


ModuleNotFoundError: No module named 'pyvis'

In [ ]:
import plotly.express as px
import pandas as pd

# Exemple de données
df = pd.DataFrame({
    "année": [2010, 2010, 2011, 2011, 2012, 2012],
    "catégorie": ["A", "B", "A", "B", "A", "B"],
    "valeur": [10, 15, 12, 18, 14, 20]
})

fig = px.bar(df, x="catégorie", y="valeur", animation_frame="année",
             range_y=[0, 25], title="Barplot animé par année")

fig.show()


In [2]:
# 📦 INSTALLATION
!pip install flashtext openpyxl

  Preparing metadata (setup.py) ... done
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9300 sha256=c3a33ed9ef97e3a6fa072dbd30af6f0f02eaf0861f1da2b1400ff43924d67ac7
  Stored in directory: /root/.cache/pip/wheels/49/20/47/f03dfa8a7239c54cbc44ff7389eefbf888d2c1873edaaec888
Successfully built flashtext


In [ ]:
# OPTIMISER AVEC ThreadPoolExecutor

In [8]:
# 📚 IMPORTS
import pandas as pd
import os
import zipfile
from collections import defaultdict
from io import BytesIO
from flashtext import KeywordProcessor
from tqdm import tqdm

def upload_file(prompt):
    """Helper function to handle file uploads"""
    print(prompt)
    try:
        from google.colab import files
        uploaded = files.upload()
        return list(uploaded.keys())[0]
    except ImportError:
        raise ImportError("This script requires Google Colab environment to run")

def process_file(filepath, archive, kp):
    """Process individual Excel file"""
    filename = os.path.basename(filepath)
    try:
        df = pd.read_excel(filepath, engine='openpyxl')

        # Vérification spécifique pour "Adresse (AD)"
        cols_lower = [col.strip().lower() for col in df.columns]

        # Recherche de la colonne "Adresse (AD)" ou "Affiliation"
        aff_col = None
        if any("adresse (ad)" in col.lower() for col in df.columns):
            # Trouver la colonne "Adresse (AD)" (insensible à la casse)
            for col in df.columns:
                if "adresse (ad)" in col.lower():
                    aff_col = col
                    break
            # Renommer en "Affiliation"
            df.rename(columns={aff_col: 'Affiliation'}, inplace=True)
        elif 'affiliation' in cols_lower:
            # Si "Affiliation" existe déjà, on l'utilise
            aff_col = df.columns[cols_lower.index('affiliation')]
        else:
            print(f"❌ Ni 'Affiliation' ni 'Adresse (AD)' trouvées dans {filename}, fichier ignoré.")
            return None

        # Add PMID if missing
        if 'PMID' not in df.columns:
            df['PMID'] = range(1, len(df) + 1)

        # Detect countries in Affiliation
        def detecter_pays_flashtext(texte):
            """Detect countries in text using flashtext"""
            if pd.isna(texte):
                return set()
            keywords_found = kp.extract_keywords(str(texte))
            return set(keywords_found) if keywords_found else set()

        df["Pays_detectés"] = df["Affiliation"].apply(detecter_pays_flashtext)
        df["Pays_detectés"] = df["Pays_detectés"].apply(
            lambda s: "; ".join(sorted(s)) if s else ""
        )

        # Add country_of_authors column
        df["country_of_authors"] = df["Pays_detectés"]

        # Save to buffer
        buffer = BytesIO()
        with pd.ExcelWriter(buffer, engine='openpyxl') as writer:
            df.to_excel(writer, index=False)

        # Write to ZIP archive
        archive.writestr(filename, buffer.getvalue())

        return f"✅ Traité et ajouté au ZIP : {filename}"

    except Exception as e:
        return f"❌ Erreur lors du traitement de {filename}: {str(e)}"

# Main execution
try:
    # 📥 TÉLÉVERSEMENT DES FICHIERS
    zip_filename = upload_file("➡️ Téléversez le fichier ZIP contenant les fichiers .xlsx…")
    list_country = upload_file("➡️ Téléversez le fichier Excel des pays (colonnes : pays, pays_variants)…")

    # Constants
    INPUT_DIR = "extracted_files"
    OUTPUT_DIR = "fichier_des_pays"
    ZIP_OUTPUT_PATH = "resultats_pays_detectes.zip"

    # 📂 EXTRACTION DU ZIP
    os.makedirs(INPUT_DIR, exist_ok=True)
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall(INPUT_DIR)

    # 🔍 CHARGEMENT DES FICHIERS EXCEL DANS LE DOSSIER
    excel_files = [
        os.path.join(root, file)
        for root, _, files_ in os.walk(INPUT_DIR)
        for file in files_
        if file.lower().endswith((".xlsx", ".xls"))
    ]

    if not excel_files:
        raise FileNotFoundError("❌ Aucun fichier Excel (.xlsx ou .xls) trouvé dans le dossier.")

    # 📄 LECTURE DU FICHIER DE PAYS
    df_pays = pd.read_excel(list_country, engine='openpyxl')
    df_pays.columns = df_pays.columns.str.strip()

    # 🧠 CRÉATION DU DICTIONNAIRE DE VARIANTS DES PAYS
    dict_pays_variants = {
        str(row.pays).strip(): [
            v.strip() for v in str(row.pays_variants).replace(";", ",").split(",") if v.strip()
        ]
        for row in df_pays.itertuples(index=False)
    }

    # 🔍 INITIALISATION DU DICTIONNAIRE DE VARIANTES AVEC LOWERCASE
    country_variants = defaultdict(set)
    for fr_name, variants in dict_pays_variants.items():
        country_variants[fr_name].add(fr_name.lower())
        for variant in variants:
            country_variants[fr_name].add(variant.lower())

    # Initialisation KeywordProcessor
    kp = KeywordProcessor(case_sensitive=False)
    for country, variants in country_variants.items():
        for variant in variants:
            kp.add_keyword(variant, country)

    # Create output directory
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # Process files sequentially (without ThreadPoolExecutor)
    with zipfile.ZipFile(ZIP_OUTPUT_PATH, mode='w', compression=zipfile.ZIP_DEFLATED) as archive:
        for filepath in tqdm(excel_files, desc="🔄 Traitement des fichiers", total=len(excel_files)):
            result = process_file(filepath, archive, kp)
            if result:
                print(result)

    print(f"\n🎉 Tous les fichiers ont été traités et archivés dans : {ZIP_OUTPUT_PATH}")

    # Download the result in Colab
    try:
        from google.colab import files
        files.download(ZIP_OUTPUT_PATH)
    except ImportError:
        print(f"Téléchargement impossible. Le fichier est disponible à : {os.path.abspath(ZIP_OUTPUT_PATH)}")

except Exception as e:
    print(f"Une erreur est survenue : {str(e)}")


➡️ Téléversez le fichier ZIP contenant les fichiers .xlsx…


Saving resultats_entites_viral_setp2.zip to resultats_entites_viral_setp2 (5).zip
➡️ Téléversez le fichier Excel des pays (colonnes : pays, pays_variants)…


Saving liste_de_pays.xlsx to liste_de_pays (5).xlsx


🔄 Traitement des fichiers:   6%|▌         | 3/54 [00:01<00:14,  3.51it/s]

✅ Traité et ajouté au ZIP : Sudan_entites.xlsx
✅ Traité et ajouté au ZIP : South Sudan_entites.xlsx
✅ Traité et ajouté au ZIP : Botswana_entites.xlsx
✅ Traité et ajouté au ZIP : Zambia_entites.xlsx


...

✅ Traité et ajouté au ZIP : Morocco_entites.xlsx
✅ Traité et ajouté au ZIP : Cabo Verde_entites.xlsx


🔄 Traitement des fichiers: 100%|██████████| 54/54 [01:02<00:00,  1.15s/it]

✅ Traité et ajouté au ZIP : Togo_entites.xlsx
✅ Traité et ajouté au ZIP : Djibouti_entites.xlsx

🎉 Tous les fichiers ont été traités et archivés dans : resultats_pays_detectes.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# 📚 IMPORTS
import pandas as pd
import os
import zipfile
from google.colab import files
from collections import defaultdict
from io import BytesIO

# 📥 TÉLÉVERSEMENT DES FICHIERS
print("➡️ Téléversez le fichier ZIP contenant les fichiers .xlsx…")
zip_filename = list(files.upload().keys())[0]

print("➡️ Téléversez le fichier Excel des pays (colonnes : pays, pays_variants)…")
list_country = list(files.upload().keys())[0]

# 📂 EXTRACTION DU ZIP
input_dir = "extracted_files"
os.makedirs(input_dir, exist_ok=True)
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(input_dir)

# 🔍 CHARGEMENT DES FICHIERS EXCEL DANS LE DOSSIER
excel_files = [
    os.path.join(root, file)
    for root, _, files_ in os.walk(input_dir)
    for file in files_
    if file.lower().endswith(".xlsx")
]
if not excel_files:
    raise Exception("❌ Aucun fichier .xlsx trouvé dans le dossier.")

# 📄 LECTURE DU FICHIER DE PAYS
df_pays = pd.read_excel(list_country, engine='openpyxl')
df_pays.columns = df_pays.columns.str.strip()  # Nettoyage des noms de colonnes

# 🧠 CRÉATION DU DICTIONNAIRE DE VARIANTS DES PAYS
dict_pays_variants = {
    str(row.pays).strip(): [
        v.strip() for v in str(row.pays_variants).replace(";", ",").split(",") if v.strip()
    ]
    for row in df_pays.itertuples(index=False)
}


➡️ Téléversez le fichier ZIP contenant les fichiers .xlsx…


Saving fichiers_virus_annotés.zip to fichiers_virus_annotés.zip
➡️ Téléversez le fichier Excel des pays (colonnes : pays, pays_variants)…


Saving liste_de_pays.xlsx to liste_de_pays.xlsx


In [ ]:
from flashtext import KeywordProcessor
from tqdm import tqdm  # Barre de progression

# 🔍 INITIALISATION DU DICTIONNAIRE DE VARIANTES AVEC LOWERCASE
country_variants = defaultdict(set)
for fr_name, variants in dict_pays_variants.items():
    country_variants[fr_name].add(fr_name.lower())
    for variant in variants:
        country_variants[fr_name].add(variant.lower())

# --- Initialisation KeywordProcessor ---
kp = KeywordProcessor(case_sensitive=False)
for country, variants in country_variants.items():
    for variant in variants:
        kp.add_keyword(variant, country)

def detecter_pays_flashtext(texte):
    return set(kp.extract_keywords(str(texte))) if pd.notna(texte) else set()

output_dir = "fichier_des_pays"
os.makedirs(output_dir, exist_ok=True)

zip_output_path = "resultats_pays_detectes.zip"

with zipfile.ZipFile(zip_output_path, mode='w', compression=zipfile.ZIP_DEFLATED) as archive:
    for filepath in tqdm(excel_files, desc="🔄 Traitement des fichiers"):
        filename = os.path.basename(filepath)
        print(f"🔎 Traitement de : {filename}")

        df = pd.read_excel(filepath, engine='openpyxl')
        cols_lower = [col.strip().lower() for col in df.columns]

        # Renommage 'adresse' en 'affiliation' si nécessaire
        if 'affiliation' not in cols_lower:
            if 'adresse' in cols_lower:
                original_col = df.columns[cols_lower.index('adresse')]
                df.rename(columns={original_col: 'affiliation'}, inplace=True)
                print(f"✅ Colonne 'adresse' renommée en 'affiliation' dans {filename}")
            else:
                print(f"❌ Ni 'affiliation' ni 'adresse' trouvée dans {filename}, fichier ignoré.")
                continue
        else:
            original_col = df.columns[cols_lower.index('affiliation')]
            if original_col != 'affiliation':
                df.rename(columns={original_col: 'affiliation'}, inplace=True)

        # Ajout colonne PMID si absente
        if 'PMID' not in df.columns:
            df['PMID'] = range(1, len(df) + 1)

        # Détection pays dans 'affiliation'
        df["Pays_detectés"] = df["affiliation"].apply(detecter_pays_flashtext)
        df["Pays_detectés"] = df["Pays_detectés"].apply(lambda s: "; ".join(sorted(s)) if s else "")

        # Ajout colonne country_of_authors identique à Pays_detectés
        df["country_of_authors"] = df["Pays_detectés"]

        # Sauvegarde dans buffer mémoire pour ajout ZIP
        buffer = BytesIO()
        df.to_excel(buffer, index=False, engine='openpyxl')
        buffer.seek(0)

        archive.writestr(filename, buffer.read())
        print(f"✅ Ajouté au ZIP : {filename}")

print(f"\n🎉 Tous les fichiers ont été traités et archivés dans : {zip_output_path}")

# Pour Google Colab, tu peux ajouter en fin de script :
try:
    from google.colab import files
    files.download(zip_output_path)
except ImportError:
    pass


🔄 Traitement des fichiers:   0%|          | 0/54 [00:00<?, ?it/s]

🔎 Traitement de : Tanzania_entites.xlsx
✅ Colonne 'adresse' renommée en 'affiliation' dans Tanzania_entites.xlsx


🔄 Traitement des fichiers:   2%|▏         | 1/54 [00:03<02:51,  3.23s/it]

✅ Ajouté au ZIP : Tanzania_entites.xlsx
🔎 Traitement de : Libya_entites.xlsx
✅ Colonne 'adresse' renommée en 'affiliation' dans Libya_entites.xlsx


🔄 Traitement des fichiers:   4%|▎         | 2/54 [00:03<01:19,  1.53s/it]

...

✅ Ajouté au ZIP : Madagascar_entites.xlsx
🔎 Traitement de : Equatorial Guinea_entites.xlsx
✅ Colonne 'adresse' renommée en 'affiliation' dans Equatorial Guinea_entites.xlsx
✅ Ajouté au ZIP : Equatorial Guinea_entites.xlsx
🔎 Traitement de : Morocco_entites.xlsx
✅ Colonne 'adresse' renommée en 'affiliation' dans Morocco_entites.xlsx


🔄 Traitement des fichiers: 100%|██████████| 54/54 [01:33<00:00,  1.73s/it]

✅ Ajouté au ZIP : Morocco_entites.xlsx

🎉 Tous les fichiers ont été traités et archivés dans : resultats_pays_detectes.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>